# Calculator

In [1]:
import pandas as pd
import json
from google_currency import convert 

In [2]:
df=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/country_currency.csv')
df.head()

,Country,last_value,Currency,Code
0,aruba,1.350690,Aruban Florin,AWG
1,afghanistan,17.914483,Afghani,AFN
2,angola,146.167168,Kwanza,AOA
3,albania,41.342391,Lek,ALL
4,united arab emirates,2.258503,UAE Dirham,AED


In [ ]:
def get_adjustment(Country)
for e in df['ppp_todollar']
    if df['ppp_todollar'][e] > 1:
        df['adjustment'] == df['ppp_todollar'][e] -1
    elif df['ppp_todollar'][e] < 1:
        df['adjustment'] == -(1 - df['ppp_todollar'][e])
    else:
        df['adjustment'] == 0.0



https://pypi.org/project/google-currency/

!pip install google-currency

In [21]:
df[df.Country == 'spain']

,Country,last_value,Currency,Code
57,spain,0.624484,Euro,EUR


# PPP to dollar

In [ ]:
df[ppp_todollar]=

In [3]:
def ppp_to_dollar(Code, last_value):
    conversion=convert(Code, 'usd', last_value)
    result = json.loads(conversion)
    return result['amount']

In [10]:
ppp_to_dollar('ZWL', 0.624484)

Invalid currency codes passed in parameters, original exception message is -> 'ZWL'


0

crear nueva columna y los argumentos de entrada sean code y last value como columnas

In [5]:
df['ppp_todollar'] = df.apply(lambda x: ppp_to_dollar(df['Code'], df['last_value']), axis=1)

TypeError: currency_from should be of type str, passed <class 'pandas.core.series.Series'>

In [4]:
df['ppp_todollar']=df.apply(lambda x: ppp_to_dollar(x['Code'], x['last_value']))

KeyError: 'Code'

In [6]:
apply = df.apply(
         lambda x: ppp_to_dollar(x['Code'], x['last_value']),
         axis=1)
df['ppp_todollar']  = apply
print(df)

Invalid currency codes passed in parameters, original exception message is -> 'BOV'
Invalid currency codes passed in parameters, original exception message is -> 'CHE'
Invalid currency codes passed in parameters, original exception message is -> 'CHW'
Invalid currency codes passed in parameters, original exception message is -> 'COU'
Invalid currency codes passed in parameters, original exception message is -> 'CUC'
Unable to convert currency, failed to fetch results from Google
Invalid currency codes passed in parameters, original exception message is -> 'ERN'
Invalid currency codes passed in parameters, original exception message is -> 'GHS'
Invalid currency codes passed in parameters, original exception message is -> 'MXV'
Invalid currency codes passed in parameters, original exception message is -> 'MNT'
Invalid currency codes passed in parameters, original exception message is -> 'MZN'
Invalid currency codes passed in parameters, original exception message is -> 'RON'
Invalid curr

In [14]:
df

,Country,last_value,Currency,Code,ppp_todollar
0,aruba,1.350690,Aruban Florin,AWG,0.75
1,afghanistan,17.914483,Afghani,AFN,0.23
2,angola,146.167168,Kwanza,AOA,0.23
3,albania,41.342391,Lek,ALL,0.41
4,united arab emirates,2.258503,UAE Dirham,AED,0.61
...,...,...,...,...,...
183,vanuatu,108.104856,Vatu,VUV,0
184,samoa,1.662674,Tala,WST,0
185,south africa,6.652566,Rand,ZAR,0.48
186,zambia,4.641657,Zambian Kwacha,ZMW,0.21
